In [1]:
%config IPCompleter.greedy=True
import pandas as pd
from datasets import load_from_disk

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


# Betöltés

betöltjük a klasszifikációs adathalmazt

In [2]:
dataset = load_from_disk('datasets/base/classification_dataset')

# Adathalmaz felfedezése

Látható, hogy három split-ből áll az adathalmaz

In [3]:
{split: len(dataset[split]) for split in dataset}

{'train': 43060, 'validation': 6598, 'test': 3618}

A label adja meg, hogy adott cikk korrupciós témájú

In [4]:
dataset.unique('label')

{'train': [0, 1], 'validation': [1, 0], 'test': [0, 1]}

A validációs halmaz csak kézzel annotált mintákat tartalmaz

In [5]:
dataset.unique('is_hand_annoted')

{'train': [False, True], 'validation': [True, False], 'test': [True]}

## Oszlopok magyarázata:

`text`: cikk törzse

`title`: hírportál által adott cím

`description`: hírportál által adott lead (kis bevezető/összefoglaló az elején)

`keywords`: hírportál címkék (nem mindig van és nem mindig értelmes)

`label`: korrupcióról szól-e a cikk (típus: int, 1 ha igen, 0 egyébként)

`url`: cikk url-je

`authors`: cikk szerzői, címkékhez hasonlóan, nem minden esetben van meg

`date`: cikk megjelenésének ideje, változó pontossággal és ritkán értelmetlen

`kmonitor_description`: csak korrupciós cikkek esetében, k-monitor adatbázisában tárolt leírás (sokszor egyezik a rendes leírással)

`kmonitor_title`: ugyanez címmel

`kmonitor_tags`: ez a keywords-el ellentétben K-Monitoros önkéntesek általi címkézés, meghatározott "címke halmazból" (ebben vannak még helyszínek, amiket people-höz és institutes-hoz hasonlóan szét lehetne (kéne) szedni egy places oszlopba)

`institutes`: cikkben megjelenő intézmények, K-Monitoros gyűjtés

`people`: ugyanez személyekkel

`is_hand_annoted`: azt jelöli, hogy adott cikket ember, vagy egy nyelvi modell annotálta, pozitív minták esetében mindig True

In [6]:
dataset['train'].features

{'text': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'description': Value(dtype='string', id=None),
 'keywords': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'label': Value(dtype='int64', id=None),
 'url': Value(dtype='string', id=None),
 'authors': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'date': Value(dtype='string', id=None),
 'kmonitor_description': Value(dtype='string', id=None),
 'kmonitor_title': Value(dtype='string', id=None),
 'kmonitor_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'kmonitor_institutes': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'kmonitor_people': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'is_hand_annoted': Value(dtype='bool', id=None),
 'kmonitor_places': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'kmonitor_collections': Sequence(feature=Value(dtype='string'

Keressünk helyreigazításokat!

In [7]:
dataset['train'].filter(lambda article: 'helyreigazítás' in article['title'].lower() or 'helyreigazítás' in article['keywords'])['title']

Filter:   0%|          | 0/43060 [00:00<?, ? examples/s]

['Helyreigazítás',
 'Helyreigazítás',
 'Helyreigazítás',
 'Helyreigazítás',
 'Belföld: Helyreigazítás',
 'Helyreigazítás',
 'Helyreigazítás - Földet is nyert a trafikos fideszes',
 'Nincs tovább: a Kúria elutasította a TEK helyreigazítási kérelmét',
 'Helyreigazítás a TV2 közleménye alapján',
 'Helyreigazítás',
 'Helyreigazítás',
 'Belföld: Bajai kamuvideó: Harangozó helyreigazítást kér',
 'Helyreigazítás',
 'Helyreigazítás az Országos Hulladékgazdálkodási Ügynökség ügyében',
 'Elfogadták a médiaalkotmányt',
 'Belföld: Helyreigazítás',
 'Helyreigazítás a FitDent Kft. ügyében',
 'Helyreigazítás',
 'Belföld: BKV-ügy: Bruck Gábor helyreigazítást kér',
 'Jogerősen megnyertük a Legfőbb Ügyészség által indított helyreigazítási pert a Quaestor-ügyben',
 'Belföld: Helyreigazítás',
 'Na, honnan jött helyreigazítási kérelem egy állami szervezet nevében?',
 'Helyreigazítás',
 'Helyreigazítás',
 'Gyurcsány nem kap helyreigazítást',
 'Helyreigazítás',
 'Rogán ügyvéd barátja annyi helyreigazítási pe

# Pandas

Az adathalmazt átalakítjuk pandas objektumokká a további feldolgozás egyszerűsítése miatt

In [8]:
dfs = {split: dataset[split].to_pandas() for split in dataset.keys()}

In [9]:
dfs['test']

,text,title,description,keywords,label,url,authors,date,kmonitor_description,kmonitor_title,kmonitor_tags,kmonitor_institutes,kmonitor_people,is_hand_annoted,kmonitor_places,kmonitor_collections
0,A TV2 honlapjára tegnap délután került ki az a...,Új reality indul a TV2-n,"Magabiztos vagy, mint Berki Krisztián, sztárna...","[tv2, való világ, vv6, édes élet]",0,https://24.hu/szorakozas/2013/11/26/uj-reality...,[],2013-11-26 00:00:00,None,None,[],[],[],True,[],[]
1,"A világ szar hely, hiába hívják Hawaii-nak, és...",Elbocsátó szép üzenet,Elbocsátó szép üzenet - George Clooney Oscar-d...,"[george clooney, alexander payne, oscar-díj, h...",0,http://index.hu/kultur/cinematrix/hirblog/2012...,[],2012-02-22 00:00:00,None,None,[],[],[],True,[],[]
2,Gyurcsány Ferencet és Jánosi Györgyöt november...,Belföld: Novemberre idézi be a bíró Gyurcsány ...,Gyurcsány Ferencet és Jánosi Györgyöt november...,[zuschlag],1,https://nol.hu/belfold/novemberre_idezi_be_a_b...,[Angyal Gábor],2009-10-19 13:42:19+02:00,Gyurcsány Ferencet és Jánosi Györgyöt november...,Novemberre idézi be a bíró Gyurcsány Ferencet,[támogatás],"[Fiatal Baloldal, Sportminisztérium]","[Benedek András, Gyurcsány Ferenc, Jánosi Györ...",True,[],[Zuschlag-ügy]
3,"Több hónapig egyezkedett az EU a görögökkel, d...",Ősszel választás jöhet Görögországban,Ősszel választás jöhet Görögországban - Pénzt ...,"[grexit, görög válság, megállapodás, szakszerv...",0,https://index.hu/gazdasag/2015/07/05/gorogcsod...,[Jenei Miklós],2015-07-05 00:00:00,None,None,[],[],[],True,[],[]
4,A gyáli rendőrök elfogtak és lopással gyanúsít...,Jogsi nélkül lopkodta az autókat a budapesti t...,Jogsi nélkül lopkodta az autókat a budapesti t...,"[kocsilopás, lopáskár, tolvaj, autó, budapest,...",0,https://index.hu/belfold/2022/09/30/auto-tolva...,"[Varga Csenge Virág, - Vcsv -]",2022-09-30 00:00:00,None,None,[],[],[],True,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,MLSZ-figyelmeztetés a szurkolóknak: most már t...,MLSZ-figyelmeztetés a szurkolóknak: most már t...,A varsói mérkőzésen nem nyit ki a vendégszekto...,"[mlasz, szurkoló, fifa, varsó, magyar válogato...",0,https://infostart.hu/sport/2021/11/11/mlsz-fig...,[],2021-11-11 00:00:00,None,None,[],[],[],True,[],[]
3614,Az FATF (Financial Action Task Force) elnöke C...,Nyáron kerülhet le Magyarország a pénzmosási f...,Az FATF (Financial Action Task Force) elnöke C...,[],1,https://origo.hu/gazdasag/hirek/20020314nyaron...,[Origo],2002-04-25 15:11:00,Az FATF (Financial Action Task Force) elnöke C...,Nyáron kerülhet le Magyarország a pénzmosási f...,[pénzmosás],[],[],True,[belföld],[]
3615,A rejtélyes oslói sorozatgyilkos legnagyobb ál...,Egy cuki kismacskától is előbb rémülünk halálr...,A rejtélyes oslói sorozatgyilkos legnagyobb ál...,"[film, hóember, jo nesbo, krimi, mozi, skandin...",0,http://24.hu/kultura/2017/10/13/egy-cuki-kisma...,[],2017-10-13 00:00:00,None,None,[],[],[],True,[],[]
3616,Adócsalás? Megkezdődött Olmert pere\nIzraelben...,MNO Magyar Nemzet Online,Adócsalás? Megkezdődött Olmert pereMNO - DNIzr...,"[cikk, képriport, képek, kép, anyag, elemzés, ...",1,http://mno.hu/portal/665351,[],None,Izraelben megkezdődött az Ehud Olmert volt kor...,Adócsalás? Megkezdődött Olmert pere,[],[],[],True,[Izrael],[]


train: 62% pozitív (korrupciós) és 38% negatív minta

validation: 51.68% pozitív (korrupciós) és 48.31% negatív

test: 50-50%

In [10]:
for split in dataset.keys():
    print(split, dfs[split].groupby(['label']).size(), sep='\n', end='\n\n')

train
label
0    16399
1    26661
dtype: int64

validation
label
0    3188
1    3410
dtype: int64

test
label
0    1898
1    1720
dtype: int64



In [11]:
full_df = pd.concat(dfs.values())

In [12]:
full_df['kmonitor_tags'].apply(pd.Series).unstack().reset_index(drop=True).dropna().value_counts()

klientúra                            4417
közbeszerzés                         3814
önkormányzat                         3550
ingatlan                             3499
támogatás                            3467
                                     ... 
megbízhatósági vizsgálat                4
NVSZ-vizsgálat                          4
kárpótlási jegy                         3
traffipax                               3
közfeladati helyzettel visszaélés       3
Length: 247, dtype: int64

In [13]:
full_df['authors'].apply(pd.Series).unstack().reset_index(drop=True).dropna().value_counts()

Hvg Kiadó Zrt.                        5642
Erdélyi Katalin                       1662
Origo                                 1120
Kis Zoltán                             941
Hont András                            832
                                      ... 
Maráczi Tamás                            1
Hanthy Kinga                             1
Dessewffy Tibor                          1
Parászka Boróka                          1
Vagy Éppen A Leveleit Nem Veszi Át       1
Length: 3088, dtype: int64

~7500 cikk megjelenési ideje hiányzik

In [14]:
full_df['date'].isna().sum()

7580